In [2]:
import pandas as pd


In [3]:
df=pd.read_csv(r"C:\Users\2024\Desktop\RFM Analysis\data\OnlineRetail.csv", encoding="unicode_escape")

In [4]:
#drop rows without customer
df=df.dropna(subset=["CustomerID"])
#remove return and cancel orders
df=df[df["Quantity"]>0]


In [5]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [6]:
#Transaction amount = quantity x unit price
df["Amount"] = df["Quantity"] * df["UnitPrice"]

In [7]:

#Aggregation

transactions = df.groupby(["CustomerID","InvoiceNo","InvoiceDate"], as_index=False)["Amount"].sum()

In [8]:
#Rename column to standardized names
transactions.rename(columns={
    "CustomerID":"customer_id",
    "InvoiceNo":"order_id",
    "InvoiceDate":"order_date"
},inplace=True)

In [9]:
transactions.head()
transactions.to_csv("clean_transactions.csv",index=False)

In [10]:
#Set a reference date
#Recency is measured relative to a fixed today date
now=df["InvoiceDate"].max() + pd.Timedelta(days=1)

In [11]:
#Grouping by customer to compute RFM

rfm= df.groupby("CustomerID").agg({
    "InvoiceDate" : lambda x: (now -x.max()).days, #Recency
    #x.max()  finds the most recvent purchase date, now - x(max) calculates the time difference between today and last purchase date
    "InvoiceNo":"nunique" ,#Frequency, counts the number of unique orders for that custoomer
    "Amount":"sum" #Monetary, adds up spend across all invoices
}).reset_index()

In [12]:
#Rename columns

rfm.rename(columns={
    "InvoiceDate":"Recency",
    "InvoiceNo":"Frequency",
    "Amount":"Monetary"
},inplace=True)

In [13]:
rfm.head()
rfm.to_csv("rfm.csv",index=False)